In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
from skimage import io
from google.colab.patches import cv2_imshow
from IPython.display import clear_output

In [ ]:
drive.mount('/content/drive')
url = "/content/drive/MyDrive/smart.mp4"
cap = cv2.VideoCapture(url)
out = cv2.VideoWriter('/content/drive/MyDrive/test-smart.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 30.0, (int(cap.get(3)),int(cap.get(4))))

first_frame = []

while True:
  ret, frame = cap.read()

  if not ret:
    break;

  if not len(first_frame):
    first_frame = frame
    continue
  d = cv2.absdiff(first_frame,frame)

  grey=cv2.cvtColor(d,cv2.COLOR_BGR2GRAY)

  blur = cv2.GaussianBlur(grey, (15, 15), 0)
  th = cv2.threshold(blur, 25, 255, cv2.THRESH_BINARY)[1]
  dilated = cv2.dilate(th, None, iterations=2)
  contours, hierachy = cv2.findContours(dilated, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

  height, width, _ = frame.shape
  min_x, min_y = width, height
  max_x = max_y = 0
  try:
    hierachy = hierachy[0]
  except:
    hierachy = []

  for contour, hier in zip(contours, hierachy):
    (x, y, w, h) = cv2.boundingRect(contour)
    min_x, max_x = min(x, min_x), max(x+w, max_x)
    min_y, max_y = min(y, min_y), max(y+h, max_y)
    if w > 80 and h > 80:
        cv2.rectangle(first_frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
  if max_x - min_x > 0 and max_y - min_y > 0:
      cv2.rectangle(first_frame, (min_x, min_y), (max_x, max_y), (255, 0, 0), 2)

  clear_output(wait=True)
  cv2_imshow(first_frame)
  out.write(first_frame)
  first_frame = frame

cap.release()
out.release()

# Новый раздел